In [1]:
!git clone 'https://github.com/shaheer1995/deep_Mahalanobis_detector-V2'

Cloning into 'deep_Mahalanobis_detector-V2'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 37 (delta 4), reused 37 (delta 4), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [3]:
%cd deep_Mahalanobis_detector-V2/
%mkdir data
%cd data

from google.colab import drive
drive.mount('/content/drive')

[Errno 2] No such file or directory: 'deep_Mahalanobis_detector-V2/'
/content/deep_Mahalanobis_detector-V2/data
/content/deep_Mahalanobis_detector-V2/data/data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/My\ Drive/Research/OOD_Detector/Data
!ls
!cp Imagenet_resize.tar.gz /content/deep_Mahalanobis_detector-V2/data/
%cd /content/deep_Mahalanobis_detector-V2/data/
!ls
!tar -xvf Imagenet_resize.tar.gz

Streaming output truncated to the last 5000 lines.
Imagenet_resize/Imagenet_resize/5134.jpg
Imagenet_resize/Imagenet_resize/9125.jpg
Imagenet_resize/Imagenet_resize/2210.jpg
Imagenet_resize/Imagenet_resize/2041.jpg
Imagenet_resize/Imagenet_resize/998.jpg
Imagenet_resize/Imagenet_resize/1813.jpg
Imagenet_resize/Imagenet_resize/5445.jpg
Imagenet_resize/Imagenet_resize/3117.jpg
Imagenet_resize/Imagenet_resize/5029.jpg
Imagenet_resize/Imagenet_resize/4136.jpg
Imagenet_resize/Imagenet_resize/7152.jpg
Imagenet_resize/Imagenet_resize/7513.jpg
Imagenet_resize/Imagenet_resize/3113.jpg
Imagenet_resize/Imagenet_resize/392.jpg
Imagenet_resize/Imagenet_resize/4996.jpg
Imagenet_resize/Imagenet_resize/1516.jpg
Imagenet_resize/Imagenet_resize/9175.jpg
Imagenet_resize/Imagenet_resize/669.jpg
Imagenet_resize/Imagenet_resize/1521.jpg
Imagenet_resize/Imagenet_resize/9835.jpg
Imagenet_resize/Imagenet_resize/5050.jpg
Imagenet_resize/Imagenet_resize/4139.jpg
Imagenet_resize/Imagenet_resize/2386.jpg
Imagenet_

In [5]:
%cd /content/drive/My\ Drive/Research/OOD_Detector/Data
!cp LSUN_resize.tar.gz /content/deep_Mahalanobis_detector-V2/data/
%cd /content/deep_Mahalanobis_detector-V2/data/
!tar -xvf LSUN_resize.tar.gz

Streaming output truncated to the last 5000 lines.
LSUN_resize/LSUN_resize/5134.jpg
LSUN_resize/LSUN_resize/9125.jpg
LSUN_resize/LSUN_resize/2210.jpg
LSUN_resize/LSUN_resize/2041.jpg
LSUN_resize/LSUN_resize/998.jpg
LSUN_resize/LSUN_resize/1813.jpg
LSUN_resize/LSUN_resize/5445.jpg
LSUN_resize/LSUN_resize/3117.jpg
LSUN_resize/LSUN_resize/5029.jpg
LSUN_resize/LSUN_resize/4136.jpg
LSUN_resize/LSUN_resize/7152.jpg
LSUN_resize/LSUN_resize/7513.jpg
LSUN_resize/LSUN_resize/3113.jpg
LSUN_resize/LSUN_resize/392.jpg
LSUN_resize/LSUN_resize/4996.jpg
LSUN_resize/LSUN_resize/1516.jpg
LSUN_resize/LSUN_resize/9175.jpg
LSUN_resize/LSUN_resize/669.jpg
LSUN_resize/LSUN_resize/1521.jpg
LSUN_resize/LSUN_resize/9835.jpg
LSUN_resize/LSUN_resize/5050.jpg
LSUN_resize/LSUN_resize/4139.jpg
LSUN_resize/LSUN_resize/2386.jpg
LSUN_resize/LSUN_resize/3382.jpg
LSUN_resize/LSUN_resize/8274.jpg
LSUN_resize/LSUN_resize/1464.jpg
LSUN_resize/LSUN_resize/1271.jpg
LSUN_resize/LSUN_resize/247.jpg
LSUN_resize/LSUN_resize/36.jp

In [6]:
!pwd
%cd /content/deep_Mahalanobis_detector-V2/
%mkdir pre_trained

%cd /content/drive/My\ Drive/Research/OOD_Detector/pre_trained
!cp resnet_cifar10.pth /content/deep_Mahalanobis_detector-V2/pre_trained/
%cd /content/deep_Mahalanobis_detector-V2/pre_trained/
!ls

%cd /content/drive/My\ Drive/Research/food-101-densenet
!cp resnet50model.pth /content/deep_Mahalanobis_detector-V2/pre_trained/
%cd /content/deep_Mahalanobis_detector/pre_trained/
!ls
%cd /content/deep_Mahalanobis_detector-V2

/content/deep_Mahalanobis_detector-V2/data
/content/deep_Mahalanobis_detector-V2
/content/drive/My Drive/Research/OOD_Detector/pre_trained
/content/deep_Mahalanobis_detector-V2/pre_trained
resnet_cifar10.pth
/content/drive/My Drive/Research/food-101-densenet
[Errno 2] No such file or directory: '/content/deep_Mahalanobis_detector/pre_trained/'
/content/drive/My Drive/Research/food-101-densenet
classes.txt  deep_Mahalanobis_detector-V2  resnet50model.pth
/content/deep_Mahalanobis_detector-V2


In [7]:
# Load the network

from __future__ import print_function
import argparse
import torch
import data_loader
import numpy as np
import calculate_log as callog
import models
import os
import lib_generation
from torchvision import datasets, transforms, models
from torchvision import transforms
from torch.autograd import Variable
from torch import nn

# set the path to pre-trained model and output
pre_trained_net = './pre_trained/resnet50model.pth'

torch.cuda.set_device(0)

# check the in-distribution dataset
out_dist_list = ['svhn', 'imagenet_resize', 'lsun_resize']

# load networks
# model = models.ResNet50()
# model.load_state_dict(torch.load(pre_trained_net, map_location = "cuda:" + str(0)))
# in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
# Use the same architecture as used earlier
checkpoint = torch.load(pre_trained_net,  map_location = "cuda:" + str(0))
model = models.resnet50(pretrained=False)
for param in model.parameters():
  param.requires_grad = False

    
classifier = nn.Linear(2048, 101)
model.fc = classifier

model.load_state_dict(checkpoint['model_state'], strict=False)
in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
model.cuda()

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# Check the saved model's last epoch and last validation loss
valid_loss = checkpoint["valid_loss_min"] 
num_epoch = checkpoint['epochs']


print(num_epoch)
print(valid_loss)


42
0.5302817171635014


In [10]:
# Helper function to download data and extract
import os
def get_data():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz
    print("Extraction done!")

In [11]:
%cd /content/deep_Mahalanobis_detector-V2/data/
get_data()
!ls

Streaming output truncated to the last 5000 lines.
food-101/images/cannoli/3778102.jpg
food-101/images/cannoli/2674065.jpg
food-101/images/cannoli/1169899.jpg
food-101/images/cannoli/1378086.jpg
food-101/images/cannoli/803909.jpg
food-101/images/cannoli/1067240.jpg
food-101/images/cannoli/3002709.jpg
food-101/images/cannoli/2612632.jpg
food-101/images/cannoli/3627612.jpg
food-101/images/cannoli/1798703.jpg
food-101/images/cannoli/560632.jpg
food-101/images/cannoli/512468.jpg
food-101/images/cannoli/2331523.jpg
food-101/images/cannoli/3118302.jpg
food-101/images/cannoli/1499412.jpg
food-101/images/cannoli/1557459.jpg
food-101/images/cannoli/146566.jpg
food-101/images/cannoli/96922.jpg
food-101/images/cannoli/1699562.jpg
food-101/images/cannoli/730051.jpg
food-101/images/cannoli/2177977.jpg
food-101/images/cannoli/2767028.jpg
food-101/images/cannoli/6564.jpg
food-101/images/cannoli/553447.jpg
food-101/images/cannoli/3452001.jpg
food-101/images/cannoli/1201456.jpg
food-101/images/cannoli/

In [12]:
from collections import defaultdict
# Helper method to split dataset into train and test folders
from shutil import copy
def prepare_data(filepath, src, dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [13]:
%cd /content/deep_Mahalanobis_detector-V2/data/food-101/
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
print("Creating train data...")
prepare_data('/content/deep_Mahalanobis_detector-V2/data/food-101/meta/train.txt', '/content/deep_Mahalanobis_detector-V2/data/food-101/images', 'food-101/train')

# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
print("Creating test data...")
prepare_data('/content/deep_Mahalanobis_detector-V2/data/food-101/meta/test.txt', '/content/deep_Mahalanobis_detector-V2/data/food-101/images', 'food-101/test')

/content/deep_Mahalanobis_detector-V2/data/food-101
Creating train data...

Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_

In [14]:
# Data augmentation
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(45),
                                       transforms.RandomAffine(45),
                                       transforms.ColorJitter(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

# Use 10-crop for Test Time Augmentation
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder("/content/deep_Mahalanobis_detector-V2/data/food-101/food-101/train", transform=train_transforms)
test_data = datasets.ImageFolder("/content/deep_Mahalanobis_detector-V2/data/food-101/food-101/test", transform=train_transforms)

# Using the image datasets and the tranforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 64, shuffle = True)

In [15]:
# Check how many files are in the train folder
print("Total number of samples in train folder")
!find food-101/train -type d -or -type f -printf '.' | wc -c

# Check how many files are in the test folder
print("Total number of samples in test folder")
!find food-101/test -type d -or -type f -printf '.' | wc -c

Total number of samples in train folder
75750
Total number of samples in test folder
25250


In [16]:
from torchvision import datasets

cifar10_train = torch.utils.data.DataLoader(
    datasets.CIFAR10('/media/tadenoud/DATADisk/datasets/cifar10/', train=True,download = True, transform=in_transform),
    batch_size=64, num_workers=2, pin_memory=True)
    
cifar10_val = torch.utils.data.DataLoader(
    datasets.CIFAR10('/media/tadenoud/DATADisk/datasets/cifar10/', train=False,download = True, transform=in_transform),
    batch_size=64, num_workers=2, pin_memory=True)

svhn_val = torch.utils.data.DataLoader(
    datasets.SVHN('/media/tadenoud/DATADisk/datasets/svhn', split='test',download = True, transform=in_transform),
    batch_size=64, num_workers=2, pin_memory=True)

lsun_val = torch.utils.data.DataLoader(
    datasets.SVHN('/media/tadenoud/DATADisk/datasets/lsun', split='test',download = True, transform=in_transform),
    batch_size=64, num_workers=2, pin_memory=True)

imagenet_val = torch.utils.data.DataLoader(
    datasets.SVHN('/media/tadenoud/DATADisk/datasets/imagenet', split='test',download = True, transform=in_transform),
    batch_size=64, num_workers=2, pin_memory=True)

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Extracting /media/tadenoud/DATADisk/datasets/cifar10/cifar-10-python.tar.gz to /media/tadenoud/DATADisk/datasets/cifar10/
Files already downloaded and verified


In [17]:
%cd /content/deep_Mahalanobis_detector-V2/
%mkdir output

/content/deep_Mahalanobis_detector-V2


In [18]:
import torch.nn.functional as F
def get_posterior(model, test_loader, magnitude, temperature, outf, out_flag):
    '''
    Compute the maximum value of (processed) posterior distribution - ODIN
    return: null
    '''
    criterion = nn.CrossEntropyLoss()
    model.eval()
    total = 0
    if out_flag == True:
        temp_file_name_val = '%s/confidence_PoV_In.txt'%(outf)
        temp_file_name_test = '%s/confidence_PoT_In.txt'%(outf)
    else:
        temp_file_name_val = '%s/confidence_PoV_Out.txt'%(outf)
        temp_file_name_test = '%s/confidence_PoT_Out.txt'%(outf)
        
    g = open(temp_file_name_val, 'w')
    f = open(temp_file_name_test, 'w')
    
    for data, _ in test_loader:
        total += data.size(0)
        data = data.cuda()
        data = Variable(data, requires_grad = True)
        batch_output = model(data)
            
        # temperature scaling
        outputs = batch_output / temperature
        labels = outputs.data.max(1)[1]
        labels = Variable(labels)
        loss = criterion(outputs, labels)
        loss.backward()
         
        # Normalizing the gradient to binary in {0, 1}
        gradient =  torch.ge(data.grad.data, 0)
        gradient = (gradient.float() - 0.5) * 2
        
       
        gradient.index_copy_(1, torch.LongTensor([0]).cuda(), gradient.index_select(1, torch.LongTensor([0]).cuda()) / (0.2023))
        gradient.index_copy_(1, torch.LongTensor([1]).cuda(), gradient.index_select(1, torch.LongTensor([1]).cuda()) / (0.1994))
        gradient.index_copy_(1, torch.LongTensor([2]).cuda(), gradient.index_select(1, torch.LongTensor([2]).cuda()) / (0.2010))

        tempInputs = torch.add(data.data,  -magnitude, gradient)
        outputs = model(Variable(tempInputs, volatile=True))
        outputs = outputs / temperature
        soft_out = F.softmax(outputs, dim=1)
        soft_out, _ = torch.max(soft_out.data, dim=1)
        
        for i in range(data.size(0)):
            if total <= 1000:
                g.write("{}\n".format(soft_out[i]))
            else:
                f.write("{}\n".format(soft_out[i]))
                
    f.close()
    g.close()

In [20]:
from __future__ import print_function
import argparse
import torch
import data_loader
import numpy as np
import calculate_log as callog
import models
import os
import lib_generation

from torchvision import transforms
from torch.autograd import Variable

# set the path to pre-trained model and output
pre_trained_net = model
outf = '/content/deep_Mahalanobis_detector-V2/output'
torch.cuda.manual_seed(0)

  
dataset = 'food-101'
  

# measure the performance
# # M_list = [0, 0.0005, 0.001, 0.0014, 0.002, 0.0024, 0.005, 0.01, 0.05, 0.1, 0.2]
M_list = [0.01]
# # T_list = [1, 10, 100, 1000]
T_list = [1]

base_line_list = []
ODIN_best_tnr = [0, 0, 0]
ODIN_best_results = [0 , 0, 0]
ODIN_best_temperature = [-1, -1, -1]
ODIN_best_magnitude = [-1, -1, -1]
for T in T_list:
    for m in M_list:
        magnitude = m
        temperature = T
        get_posterior(model, test_loader, magnitude, temperature, outf, True)
        out_count = 0
        print('Temperature: ' + str(temperature) + ' / noise: ' + str(magnitude)) 
      
        print('Out-distribution: ' + 'svhn') 
        get_posterior(model, svhn_val, magnitude, temperature, outf, False)
        if temperature == 1 and magnitude == 0:
            test_results = callog.metric(outf, ['PoT'])
            base_line_list.append(test_results)
        else:
            val_results = callog.metric(outf, ['PoV'])
            if ODIN_best_tnr[out_count] < val_results['PoV']['TNR']:
                ODIN_best_tnr[out_count] = val_results['PoV']['TNR']
                ODIN_best_results[out_count] = callog.metric(outf, ['PoT'])
                ODIN_best_temperature[out_count] = temperature
                ODIN_best_magnitude[out_count] = magnitude
        out_count += 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Temperature: 1 / noise: 0.01
Out-distribution: svhn


In [21]:
%cd /content/deep_Mahalanobis_detector-V2/

/content/deep_Mahalanobis_detector-V2


In [22]:
!zip -r /content/output.zip /content/deep_Mahalanobis_detector-V2/output/

  adding: content/deep_Mahalanobis_detector-V2/output/ (stored 0%)
  adding: content/deep_Mahalanobis_detector-V2/output/confidence_PoT_In.txt (deflated 71%)
  adding: content/deep_Mahalanobis_detector-V2/output/confidence_PoV_Out.txt (deflated 56%)
  adding: content/deep_Mahalanobis_detector-V2/output/confidence_PoV_In.txt (deflated 67%)
  adding: content/deep_Mahalanobis_detector-V2/output/confidence_PoT_Out.txt (deflated 59%)


In [23]:
!tar -czvf output.tar.gz output
!pwd

output/
output/confidence_PoT_In.txt
output/confidence_PoV_Out.txt
output/confidence_PoV_In.txt
output/confidence_PoT_Out.txt
/content/deep_Mahalanobis_detector-V2


In [24]:
print(base_line_list)

[]


In [25]:
print(ODIN_best_results)

[{'PoT': {'TNR': 0.2668714103382259, 'AUROC': 0.7958288429036191, 'DTACC': 0.7576293719949042, 'AUIN': 0.7063563326891354, 'AUOUT': 0.7658988214102241}}, 0, 0]


In [26]:
# print the results
mtypes = ['TNR', 'AUROC', 'DTACC', 'AUIN', 'AUOUT']
print('Baseline method: in_distribution: ' + 'food-101' + '==========')
count_out = 0
for results in base_line_list:
    print('out_distribution: '+ 'svhn')
    for mtype in mtypes:
        print(' {mtype:6s}'.format(mtype=mtype), end='')
    print('\n{val:6.2f}'.format(val=100.*results['PoT']['TNR']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['AUROC']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['DTACC']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['AUIN']), end='')
    print(' {val:6.2f}\n'.format(val=100.*results['PoT']['AUOUT']), end='')
    print('')
    count_out += 1
    
print('ODIN method: in_distribution: ' + 'food-101' + '==========')
count_out = 0
for results in ODIN_best_results:
    print('out_distribution: '+ 'svhn')
    for mtype in mtypes:
        print(' {mtype:6s}'.format(mtype=mtype), end='')
    print('\n{val:6.2f}'.format(val=100.*results['PoT']['TNR']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['AUROC']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['DTACC']), end='')
    print(' {val:6.2f}'.format(val=100.*results['PoT']['AUIN']), end='')
    print(' {val:6.2f}\n'.format(val=100.*results['PoT']['AUOUT']), end='')
    print('temperature: ' + str(ODIN_best_temperature[count_out]))
    print('magnitude: '+ str(ODIN_best_magnitude[count_out]))
    print('')
    count_out += 1


Baseline method: in_distribution: food-101==========
ODIN method: in_distribution: food-101==========
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 26.69  79.58  75.76  70.64  76.59
temperature: 1
magnitude: 0.01

out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 

TypeError: ignored